In [ ]:
import zmq
ctx = zmq.Context()
# The REQ talks to Pupil remote and receives the session unique IPC SUB PORT
pupil_remote = ctx.socket(zmq.REQ)

ip = 'localhost'  # If you talk to a different machine use its IP.
port = 50020  # The port defaults to 50020. Set in Pupil Capture GUI.

pupil_remote.connect(f'tcp://{ip}:{port}')

# Request 'SUB_PORT' for reading data
pupil_remote.send_string('SUB_PORT')
sub_port = pupil_remote.recv_string()

# Request 'PUB_PORT' for writing data
pupil_remote.send_string('PUB_PORT')
pub_port = pupil_remote.recv_string()

In [ ]:
import cv2
import numpy as np


# Assumes `sub_port` to be set to the current subscription port
subscriber = ctx.socket(zmq.SUB)
subscriber.connect(f'tcp://{ip}:{sub_port}')
subscriber.subscribe("frame.world")  # receive all gaze messages

frame = subscriber.recv_multipart()
frame_data = frame[2]
decoded_frame = cv2.imdecode(np.frombuffer(frame_data, dtype=np.uint8), cv2.IMREAD_COLOR)


In [ ]:
import inference

model = inference.load_roboflow_model("yolov8n-640")
results = model.infer(image="YOUR_IMAGE.jpg")

In [ ]:
import matplotlib.pyplot as plt

# Convert the color from BGR (OpenCV default) to RGB for proper display with matplotlib
rgb_frame = cv2.cvtColor(decoded_frame, cv2.COLOR_BGR2RGB)

# Display the image using matplotlib
plt.imshow(rgb_frame)
plt.axis('off')  # Hide axes for a cleaner display
plt.show()